In [1]:
#Reset kernel when start this notebook
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

spark = SparkSession.builder.master('local[*]').getOrCreate()

sc = spark.sparkContext

spark

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

Your runtime has 16.8 gigabytes of available RAM



In [6]:
spark.sparkContext.defaultParallelism

8

In [7]:
df = spark.read.format("parquet").option("inferSchema", "true").option("timestampFormat","yyyy-MM-dd HH:mm:ss").option("header", "true").option("mode", "DROPMALFORMED").load("content/yellow_tripdata_2017-01.parquet")

In [8]:
df.rdd.count()

9710820

In [9]:
filtered = df.rdd.filter(lambda x: x[1] is not None and x[2] is not None and x[4] is not None).filter(lambda x: (x[2] - x[1]).seconds > 0)

In [10]:
mapped = filtered.map(lambda x: (x[1].hour, (x[4]/(((x[2] - x[1]).seconds)/3600), 1)))

In [11]:
result = mapped.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

In [12]:
result.sortBy(lambda x: x[0], True).take(24)

[(0, (5414162.309641142, 315964)),
 (1, (4270126.96305057, 241135)),
 (2, (3377608.958453877, 187586)),
 (3, (2518798.78802453, 138729)),
 (4, (2196336.6165762604, 105769)),
 (5, (2320946.31090887, 103011)),
 (6, (3970573.757003464, 219829)),
 (7, (5089813.79451558, 371686)),
 (8, (5291352.552403971, 453258)),
 (9, (5289697.083458636, 450592)),
 (10, (5320581.825946025, 436412)),
 (11, (5470946.657648884, 457051)),
 (12, (5908070.528578324, 490381)),
 (13, (6219085.433525098, 495899)),
 (14, (6168205.592846137, 519463)),
 (15, (6147466.963870785, 529653)),
 (16, (6030333.506517751, 486638)),
 (17, (6325827.323365864, 540032)),
 (18, (7146001.153469369, 615740)),
 (19, (7415741.855794768, 590990)),
 (20, (7436755.1435884815, 534508)),
 (21, (7927195.430849889, 524051)),
 (22, (7831092.30898836, 495057)),
 (23, (6834603.797282512, 398047))]

In [13]:
result.map(lambda x: (x[0], x[1][0] / x[1][1] )).sortBy(lambda x: x[0], True).take(24)

[(0, 17.135377162085373),
 (1, 17.7084494704235),
 (2, 18.005655850937046),
 (3, 18.156252751944653),
 (4, 20.76540968125122),
 (5, 22.53105310024046),
 (6, 18.062101710891028),
 (7, 13.693853937236215),
 (8, 11.674041169497219),
 (9, 11.739438524116354),
 (10, 12.19164877672022),
 (11, 11.970101055787831),
 (12, 12.047918921365886),
 (13, 12.541032414917348),
 (14, 11.874196223496451),
 (15, 11.60659330518431),
 (16, 12.391826175756417),
 (17, 11.713800892106143),
 (18, 11.605549669453614),
 (19, 12.547998876114262),
 (20, 13.913271912840372),
 (21, 15.1267632937441),
 (22, 15.818566971052546),
 (23, 17.170343696303483)]